# Neural Time Series Forescasting

In [12]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as  np
import seaborn as sns #estadisticas basicas de visualizacion

sns.set(style="whitegrid", color_codes=True)

path='../2018_01_23_datos_walmart/'
df_train = pd.read_csv(path +'1_data.csv', index_col=0)

#sacar todos los Item que no sean 3,4,14,66,70,168,173,316,415,475
items=[3,4,14,40,66,70,168,173,316,415,475]
#df = df[df.Item==1]
df_train=df_train[df_train['Item'].isin(items)]#los que se encuentran en el arreglo


df_train.index = pd.to_datetime(df_train.index)
df_train.shape #dimensionalidad

df_train.columns.values.tolist()


['Local', 'Item', 'Unidades', 'Venta', 'Inventario']

In [13]:
#Agregar

In [14]:
def expand_df(df):
    data = df.copy()
    #data['month'] = data.index.month
    #data['year'] = data.index.year
    data['weekend'] = np.int32(data.index.dayofweek > 3)
    
    
    return data

df_train=expand_df(df_train)
df_train.tail()  #devolver las filas

,Local,Item,Unidades,Venta,Inventario,weekend
Fecha,,,,,,
2018-03-31,28,66,NaN,NaN,12.0,1
2018-03-31,28,173,1.0,6714.0,7.0,1
2018-03-31,28,14,1.0,5042.0,32.0,1
2018-03-31,28,415,1.0,3353.0,21.0,1
2018-03-31,28,168,2.0,16790.0,46.0,1


De manera de utilizar redes neuronales, debemos convertir las series de tiempo en un problema de aprendizaje supervisado.


In [9]:
def series_to_supervised(data, window=1, lag=1, dropnan=True):
    cols, names = list(), list()
    # Input sequence (t-n, ... t-1)
    for i in range(window, 0, -1):
        cols.append(data.shift(i))
        names += [('%s(t-%d)' % (col, i)) for col in data.columns]
    # Current timestep (t=0)
    cols.append(data)
    names += [('%s(t)' % (col)) for col in data.columns]
    # Target timestep (t=lag)
    cols.append(data.shift(-lag))
    names += [('%s(t+%d)' % (col, lag)) for col in data.columns]
    # Put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # Drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

window = 3 # t - windows
lag = 1 # t + lag
df_train = series_to_supervised(df_train, window=window, lag=lag)
df_train.columns.values.tolist()

['Local(t-3)',
 'Item(t-3)',
 'Unidades(t-3)',
 'Venta(t-3)',
 'Inventario(t-3)',
 'weekend(t-3)',
 'Local(t-2)',
 'Item(t-2)',
 'Unidades(t-2)',
 'Venta(t-2)',
 'Inventario(t-2)',
 'weekend(t-2)',
 'Local(t-1)',
 'Item(t-1)',
 'Unidades(t-1)',
 'Venta(t-1)',
 'Inventario(t-1)',
 'weekend(t-1)',
 'Local(t)',
 'Item(t)',
 'Unidades(t)',
 'Venta(t)',
 'Inventario(t)',
 'weekend(t)',
 'Local(t+1)',
 'Item(t+1)',
 'Unidades(t+1)',
 'Venta(t+1)',
 'Inventario(t+1)',
 'weekend(t+1)']

In [10]:
df_train.head()

,Local(t-3),Item(t-3),Unidades(t-3),Venta(t-3),Inventario(t-3),weekend(t-3),Local(t-2),Item(t-2),Unidades(t-2),Venta(t-2),...,Unidades(t),Venta(t),Inventario(t),weekend(t),Local(t+1),Item(t+1),Unidades(t+1),Venta(t+1),Inventario(t+1),weekend(t+1)
Fecha,,,,,,,,,,,,,,,,,,,,,
2017-01-07,6.0,66.0,6.0,40284.0,28.0,1.0,6.0,173.0,2.0,13428.0,...,2.0,13428.0,10.0,1,14.0,66.0,1.0,6714.0,26.0,1.0
2017-01-07,22.0,66.0,2.0,13428.0,73.0,1.0,22.0,173.0,4.0,26856.0,...,2.0,13428.0,72.0,1,79.0,173.0,2.0,13428.0,9.0,1.0
2017-01-07,43.0,66.0,1.0,6714.0,60.0,1.0,43.0,173.0,3.0,20142.0,...,1.0,6714.0,15.0,1,27.0,173.0,3.0,20142.0,21.0,1.0
2017-01-07,43.0,173.0,3.0,20142.0,15.0,1.0,43.0,14.0,2.0,13428.0,...,3.0,20142.0,21.0,1,5.0,66.0,4.0,26856.0,36.0,1.0
2017-01-07,43.0,14.0,2.0,13428.0,83.0,1.0,27.0,66.0,1.0,6714.0,...,4.0,26856.0,36.0,1,5.0,173.0,1.0,6714.0,15.0,1.0
